# Proyecto 1 - Analítica de textos 
## Análisis de sentimientos en películas

### Integrantes:
- Daniel Aguilera
- Vanessa Martínez
- Cristian Sánchez

### Descripción del problema
El objetivo de este notebook es realizar el desarrollo de la etapa 2 para el cual, se llevara acabo el analisis de sentimiento de las opiniones de las peliculas que se encuentran en el dataset MovieReviews.csv, para este fin se utilizara un modelo de ***random forest*** para clasificar las opiniones en positivas y negativas. 

Se usara un modelo implementado usando ***random forest*** debido a que este modelo es uno de los mas utilizados para este tipo de problemas, ademas de que es un modelo que se puede entrenar de manera rapida y que no requiere de mucha configuracion para obtener buenos resultados.

## Referencias
1. [Datos a utilizar: MovieReviews.csv](./datos/MovieReviews.csv)
2. [Diccionario de datos: DiccionarioPeliculas.xlsx](./datos/DiccionarioPeliculas.xlsx)
3. [Datos de prueba: MovieReviewsPruebas.csv](./datos/MovieReviewsPruebas.csv)
4. [Datos procesados: MovieReviewsProcesado.csv](./datos/MovieReviewsProcesado.csv)
5. [Notebook de procesamiento y entendimiento de datos](./proy_desarrollo.ipynb)
6. [Información del negocio](./datos/enunciado.pdf)

### Importar librerias

In [32]:
#Importar librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix



## Carga de datos ya procesados

In [33]:
#Cargar el dataset
df_movies = pd.read_csv('./datos/MovieReviewsProcesado.csv')

#Elimiar la columna Unnamed: 0
df_movies = df_movies.drop(['Unnamed: 0'], axis=1)

#Visualizar el dataset
df_movies.head()


,review_es,sentimiento,processed_text
0,Si está buscando una película de guerra típica...,1,si busc pelicul guerr tipic asi not aficion gu...
1,Supongo que algunos directores de películas de...,1,supon director pelicul luj sent busc abrig gra...
2,Es difícil contarle más sobre esta película si...,1,dificil contar el mas pelicul estropearlal dis...
3,"La película comienza muy lentamente, con el es...",1,pelicul comenz lent estil vid wallac napalm as...
4,Esta película es verdadera acción en su máxima...,1,pelicul verdader accion maxim expresion mejor ...


## Preparación de datos para el entrenamiento

En este paso se preparan los datos para el entrenamiento del modelo, se separan los datos en entrenamiento y prueba.

In [34]:
from sklearn.model_selection import train_test_split

#Separar los datos en train y test
features = df_movies.drop(['sentimiento'], axis=1)
labels = df_movies['sentimiento']

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.5, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)
#Visualizar la distribucion de los datos
print("Data distribution:\n- Train: {} \n- Validation: {} \n- Test: {}".format(len(y_train),len(y_val),len(y_test)))

Data distribution:
- Train: 2499 
- Validation: 1249 
- Test: 1250


## Vectorización de datos

En este paso se vectorizan los datos para poder entrenar el modelo. Se utilizará TF-IDF para vectorizar los datos, ya que este método es uno de los mas utilizados para este tipo de problemas.

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer

def vectorize_text(data, vectorizer):
    X_vector = vectorizer.transform(data)
    words = vectorizer.get_feature_names()
    X_vector_df = pd.DataFrame(X_vector.toarray())
    X_vector_df.columns = words
    
    return(X_vector_df)

In [36]:
tfidf_vector = TfidfVectorizer()
tfidf_vector_fit = tfidf_vector.fit(X_train['processed_text'])
X_train = vectorize_text(X_train['processed_text'], tfidf_vector_fit)

# vectorizar conjunto de prueba
X_test = vectorize_text(X_test['processed_text'], tfidf_vector_fit)



c:\Users\crist\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\Users\crist\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


NotFittedError: The TF-IDF vectorizer is not fitted

## Prueba de modelo

En este paso se prueba el modelo con los datos ya vectorizados y se obtiene el resultado de la predicción.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rf = RandomForestClassifier()
scores = cross_val_score(rf, X_train, y_train.values.ravel(), cv=5)
print (scores)
scores.mean()

[0.786      0.768      0.788      0.768      0.80561122]


0.783122244488978

## Busqueda de hiperparametros

En este paso se busca el mejor modelo para el problema, para esto se realizara una busqueda de hiperparametros, para esto se utilizara la libreria ***GridSearchCV*** de ***sklearn***.


In [ ]:
def print_results(results):
    print('BEST PARAMS: {}\n'.format(results.best_params_))

    means = results.cv_results_['mean_test_score']
    stds = results.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, results.cv_results_['params']):
        print('{} (+/-{}) for {}'.format(round(mean, 3), round(std * 2, 3), params))

In [ ]:
from sklearn.model_selection import GridSearchCV

rf = RandomForestClassifier()
parameters = {
    'n_estimators': [5,50,100],
    'max_depth': [2,10,20,None]
}

cv = GridSearchCV(rf, parameters)
cv.fit(X_train, y_train.values.ravel())

print_results(cv)


BEST PARAMS: {'max_depth': None, 'n_estimators': 100}

0.576 (+/-0.099) for {'max_depth': 2, 'n_estimators': 5}
0.626 (+/-0.068) for {'max_depth': 2, 'n_estimators': 50}
0.621 (+/-0.038) for {'max_depth': 2, 'n_estimators': 100}
0.613 (+/-0.042) for {'max_depth': 10, 'n_estimators': 5}
0.752 (+/-0.036) for {'max_depth': 10, 'n_estimators': 50}
0.765 (+/-0.048) for {'max_depth': 10, 'n_estimators': 100}
0.664 (+/-0.037) for {'max_depth': 20, 'n_estimators': 5}
0.756 (+/-0.023) for {'max_depth': 20, 'n_estimators': 50}
0.777 (+/-0.019) for {'max_depth': 20, 'n_estimators': 100}
0.654 (+/-0.045) for {'max_depth': None, 'n_estimators': 5}
0.765 (+/-0.058) for {'max_depth': None, 'n_estimators': 50}
0.783 (+/-0.049) for {'max_depth': None, 'n_estimators': 100}


In [ ]:
#Se identifica el mejor modelo 
cv.best_estimator_

RandomForestClassifier()

## Evaluación del modelo



In [ ]:
X_val = vectorize_text(X_val['processed_text'], tfidf_vector_fit)
X_test = vectorize_text(X_test['processed_text'], tfidf_vector_fit)

rf1 = RandomForestClassifier(n_estimators=100,max_depth=20)
rf1.fit(X_train, y_train.values.ravel())

rf2 = RandomForestClassifier(n_estimators=100,max_depth=None)
rf2.fit(X_train, y_train.values.ravel())

rf3 = RandomForestClassifier(n_estimators=5,max_depth=None)
rf3.fit(X_train, y_train.values.ravel())

c:\Users\crist\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\Users\crist\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


RandomForestClassifier(n_estimators=5)

In [ ]:
from sklearn.metrics import accuracy_score,precision_score,recall_score

for mdl in [rf1,rf2,rf3]:
    y_pred = mdl.predict(X_test)
    accuracy = round(accuracy_score(y_test, y_pred), 3)
    precision = round(precision_score(y_test, y_pred), 3)
    recall = round(recall_score(y_test, y_pred), 3)
    print('MAX DEPTH: {} / # OF EST: {} -- A: {} / P: {} / R: {}'.format(mdl.max_depth,
                                                                         mdl.n_estimators,
                                                                         accuracy,
                                                                         precision,
                                                                         recall))

c:\Users\crist\anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- 08los
- 100a
- 100bad
- 1010el
- 10casi
- ...
Feature names seen at fit time, yet now missing:
- 01mark
- 07he
- 1010esper
- 1010segund
- 10btw
- ...

  warnings.warn(message, FutureWarning)


ValueError: X has 20456 features, but RandomForestClassifier is expecting 32182 features as input.